# Open Power System Data: Renewable power plants


## Part 1: Download and processing of the original data

This script downlads and extracts the original data of renewable power plant lists from the data sources, processes and merges them. It subsequently adds the geolocation for each power plant and gives an overview over the structure of the data frame. Finally it saves the data frame.

# Table of contents 

* [1. Script setup](#1.-Script-setup)
* [2. Settings](#2.-Settings)
    * [2.1 Choose download option](#2.1-Choose-download-option)
    * [2.2 Download function](#2.2-Download-function)
    * [2.3 Setup translation dictionaries](#2.3-Setup-translation-dictionaries)
* [3. Download and process per country](#3.-Download-and-process-per-country)
    * [3.1 Germany DE](#3.1-Germany-DE)
        * [3.1.1 Download and read](#3.1.1-Download-and-read)
        * [3.1.2 Translate column names](#3.1.2-Translate-column-names)
        * [3.1.3 Add data source and missing information](#3.1.3-Add-data-source-and-missing-information)
        * [3.1.4 Merge DataFrames and choose columns](#3.1.4-Add-data-source-and-missing-information)
        * [3.1.5 Translate values and harmonize energy source](#3.1.5-Translate-values-and-harmonize-energy-source)
        * [3.1.6 Georeferencing](#3.1.6-Georeferencing)
        * [3.1.7 Save](#3.1.7-Save)
    * [3.2 Denmark DK](#3.2-Denmark-DK)
        * [3.2.1 Download and read](#3.2.1-Download-and-read)
        * [3.2.2 Add information](#3.2.2-Add-information)
        * [3.2.3 Translation](#3.2.3-Translation)
        * [3.2.4 Georeferencing](#3.2.4-Georeferencing)
        * [3.2.5 Save](#3.2.5-Save)
    * [3.3 France FR](#3.3-France-FR)
        * [3.3.1 Download and read](#3.3.1-Download-and-read)
        * [3.3.2 Add information](#3.3.2-Add-information)
        * [3.3.3 Translation](#3.3.3-Translation)
        * [3.3.4 Georeferencing](#3.3.4-Georeferencing)
        * [3.3.5 Save](#3.3.5-Save)
    * [3.4 Poland PL](#3.4-Poland-PL)
        * [3.4.1 Download and read](#3.4.1-Download-and-read)
        * [3.4.2 Add information](#3.4.2-Add-information)
        * [3.4.3 Translation](#3.4.3-Translation)
        * [3.4.4 Georeferencing](#3.4.4-Georeferencing)
        * [3.4.5 Save](#3.4.5-Save)


# 1. Script setup


In [4]:
# importing all necessary Python libraries for this Script

from collections import OrderedDict
import io
import json
import os
import subprocess
import zipfile
import posixpath
import urllib.parse
import urllib.request
import numpy as np
import pandas as pd
import requests 
import sqlite3 
import logging
import getpass

# Starting from ipython 4.3.0 logging is not directing its ouput to the out cell. It might be operating system related but 
# until the issue is fixed, we are going to use print(). 
# Issue on Github: https://github.com/ipython/ipykernel/issues/111

# Set up a log 
logger = logging.getLogger('notebook')
logger.setLevel('INFO')
nb_root_logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s'\
                              '- %(message)s',datefmt='%d %b %Y %H:%M:%S')

# Create input and output folders if they don't exist
os.makedirs('input/original_data', exist_ok=True)
os.makedirs('output', exist_ok=True)
os.makedirs('output/renewable_power_plants', exist_ok=True)

# 2. Settings

## 2.1 Choose download option
The original data can either be downloaded from the original data sources as specified below or from the opsd-Server. Default option is to download from the original sources as the aim of the project is to stay as close to original sources as possible. However, if problems with downloads e.g. due to changing urls occur, you can still run the script with the original data from the opsd_server.

In [ ]:
download_from = 'original_sources'
# download_from = 'opsd_server' 

In [ ]:
if download_from == 'opsd_server'

# While OPSD is in beta, we need to supply authentication
    password = getpass.getpass('Please enter the beta user password:')
    session = requests.session()
    session.auth = ('beta', password) 

# Specify direction to original_data folder on the opsd data server
    url_opsd = 'http://data.open-power-system-data.org/renewables_power_plants/'
    version = '2016-08-25'
    folder = '/original_data'

## 2.2 Download function

In [ ]:
def download_and_cache(url, session=None):
    """This function downloads a file into a folder called 
    original_data and returns the local filepath."""
    path = urllib.parse.urlsplit(url).path
    filename = posixpath.basename(path)
    filepath = "input/original_data/" + filename

    if not session:
        session = requests.session()

    r = session.get(url)

    # check if file exists, if not download it
    filepath = "input/original_data/" + filename
    if not os.path.exists(filepath):
        print("Downloading file: ", filename)
        r = session.get(url, stream=True)

        chuncksize = 1024
        with open(filepath, 'wb') as file:
            for chunck in r.iter_content(chuncksize):
                file.write(chunck)
    else:
        print("Using local file from", filepath)
    filepath = '' + filepath
    return filepath

## 2.3 Setup translation dictionaries

Column and value names of the original data sources will be translated to English and standardized across different sources. Standardized column names, e.g. "electrical_capacity" are required to merge data in on data frame.<br>
The column and the value translation lists are provided in the input folder of the datapackage.

In [19]:
# Get column translation list
columnnames = pd.read_csv('input/column_translation_list.csv')

In [20]:
# Get value translation list
valuenames = pd.read_csv('input/value_translation_list.csv')

In [21]:
# Create dictionnaries to harmonize columns names, values, energy source
column_dict = columnnames.set_index('original_name')['opsd_name'].to_dict()
value_dict = valuenames.set_index('original_name')['opsd_name'].to_dict()
energy_source_dict = valuenames.set_index('opsd_name')['energy_source'].to_dict()

# 3. Download and process per country

For one country after the other, the original data is downloaded, read, processed, translated, eventually georeferenced and saved. If respective files are already in the local folder, these will be utilized.
To process the provided data [pandas DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) is applied.<br>

## 3.1 Germany DE

### 3.1.1 Download and read
The data which will be processed below is provided by the following data sources:

**[Netztransparenz.de](https://www.netztransparenz.de/de/Anlagenstammdaten.htm)** - Official grid transparency platform from the German TSOs (50Hertz, Amprion, TenneT and TransnetBW).

**Bundesnetzagentur (BNetzA)** - German Federal Network Agency for Electricity, Gas, Telecommunications, Posts and Railway (Data for [roof-mounted PV power plants](http://www.bundesnetzagentur.de/cln_1422/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/ErneuerbareEnergien/Photovoltaik/DatenMeldgn_EEG-VergSaetze/DatenMeldgn_EEG-VergSaetze_node.html) and for [all other renewable energy power plants](http://www.bundesnetzagentur.de/cln_1412/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/ErneuerbareEnergien/Anlagenregister/Anlagenregister_Veroeffentlichung/Anlagenregister_Veroeffentlichungen_node.html))

In [ ]:
# point URLs to original data depending on the chosen download option
if download_from == 'original_sources':
     
    url_netztransparenz ='https://www.netztransparenz.de/de/file/Anlagenstammdaten_2015_final.zip'  
    url_bnetza ='http://www.bundesnetzagentur.de/SharedDocs/Downloads/DE/Sachgebiete/Energie/Unternehmen_Institutionen/ErneuerbareEnergien/Anlagenregister/VOeFF_Anlagenregister/2016_06_Veroeff_AnlReg.xls?__blob=publicationFile&v=1'
    url_bnetza_pv = 'https://www.bundesnetzagentur.de/SharedDocs/Downloads/DE/Sachgebiete/Energie/Unternehmen_Institutionen/ErneuerbareEnergien/Photovoltaik/Datenmeldungen/Meldungen_Aug-Mai2016.xls?__blob=publicationFile&v=2'
    
else:
    url_netztransparenz = (url_opsd + version + folder + '/Netztransparenz/' + 'Anlagenstammdaten_2015.zip')
    url_bnetza = (url_opsd + version + folder + '/BNetzA/' + '2016_06_Veroeff_AnlReg.xls')
    url_bnetza_pv = (url_opsd + version + folder + '/BNetzA/' + 'Meldungen_Aug-Mai2016.xls')

In [ ]:
# Download all data sets before processing.
if download_from == 'original_sources':
    
    netztransparenz_zip = %time zipfile.ZipFile(download_and_cache(url_netztransparenz))
    bnetza_xls = %time download_and_cache(url_bnetza)
    bnetza_pv_xls = %time download_and_cache(url_bnetza_pv)

else:
    netztransparenz_zip = %time zipfile.ZipFile(download_and_cache(url_netztransparenz, session))
    bnetza_xls = %time download_and_cache(url_bnetza, session)
    bnetza_pv_xls = %time download_and_cache(url_bnetza_pv, session)

In [ ]:
# Read TSO data from zip file
print('Reading Amprion_Anlagenstammdaten_2015.csv')
amprion_df = pd.read_csv(netztransparenz_zip.open('Amprion_Anlagenstammdaten_2015.csv'),
                         sep=';',
                         thousands='.',
                         decimal=',',
                         header=0,
                         parse_dates=[11, 12, 13, 14],
                         encoding='cp1252',
                         dayfirst=True)

print('Reading 50Hertz_Anlagenstammdaten_2015.csv')
hertz_df = pd.read_csv(netztransparenz_zip.open('50Hertz_Anlagenstammdaten_2015.csv'),
                       sep=';',
                       thousands='.',
                       decimal=',',
                       header=0,
                       parse_dates=[11, 12, 13, 14],
                       encoding='cp1252',
                       dayfirst=True)

print('Reading TenneT_Anlagenstammdaten_2015.csv')
tennet_df = pd.read_csv(netztransparenz_zip.open('TenneT_Anlagenstammdaten_2015.csv'),
                        sep=';',
                        thousands='.',
                        decimal=',',
                        header=0,
                        parse_dates=[11, 12, 13, 14],
                        encoding='cp1252',
                        dayfirst=True)

print('Reading TransnetBW_Anlagenstammdaten_2015.csv')
transnetbw_df = pd.read_csv(netztransparenz_zip.open('TransnetBW_Anlagenstammdaten_2015.csv'),
                            sep=';',
                            thousands='.',
                            decimal=',',
                            header=0,
                            parse_dates=[11, 12, 13, 14],
                            encoding='cp1252',
                            dayfirst=True,
                            low_memory=False)

In [ ]:
# Read BNetzA register
print('Reading bnetza - 2016_06_Veroeff_AnlReg.xls')
bnetza_df = pd.read_excel(bnetza_xls,
                          sheetname='Gesamtübersicht',
                          header=0,
                          converters={'4.9 Postleit-zahl': str})

# Read BNetzA-PV register
print('Reading bnetza_pv - Meldungen_Aug-Mai2016.xls')
bnetza_pv = pd.ExcelFile(bnetza_pv_xls)

# Combine all PV BNetzA sheets into one data frame
print('Concatenating bnetza_pv')
bnetza_pv_df = pd.concat(bnetza_pv.parse(sheet, skiprows=10,
                                         converters={'Anlage \nPLZ': str}
                                         ) for sheet in bnetza_pv.sheet_names)

# Drop not needed NULL "Unnamed:" column
bnetza_pv_df = bnetza_pv_df.drop(bnetza_pv_df.columns[[7]], axis=1)

### 3.1.2 Translate column names
To standardise the data frame the original column names from the German TSOs and the BNetzA wil be translated and new english column names wil be assigned to the data frame. The unique column names are required to merge the data frame.<br>
The column_translation_list is provided here as csv in the input folder. It is loaded in _2.3 Setup of translation dictionaries_.

In [ ]:
print('Translation')
amprion_df.rename(columns=column_dict, inplace=True)
hertz_df.rename(columns=column_dict, inplace=True)
tennet_df.rename(columns=column_dict, inplace=True)
transnetbw_df.rename(columns=column_dict, inplace=True)
bnetza_df.rename(columns=column_dict, inplace=True)
bnetza_pv_df.rename(columns=column_dict, inplace=True)

### 3.1.3 Add data source and missing information
All data source names and (for the BNetzA-PV data) the energy source will is added.

In [ ]:
# Add data source names to the data frames
transnetbw_df['data_source'] = 'TransnetBW'
tennet_df['data_source'] = 'TenneT'
amprion_df['data_source'] = 'Amprion'
hertz_df['data_source'] = '50Hertz'
bnetza_df['data_source'] = 'BNetzA'
bnetza_pv_df['data_source'] = 'BNetzA_PV'

# Add for the BNetzA PV data the energy source
bnetza_pv_df['energy_source'] = 'solar'

### 3.1.4 Merge DataFrames and choose columns

The individual data frames from the TSOs (Netztransparenz.de) and BNetzA will be merged together. Only some columns will be selected for further processing.

In [1]:
# Merge DataFrames
dataframes = [transnetbw_df, tennet_df, amprion_df, hertz_df, bnetza_pv_df, bnetza_df]
renewables = pd.concat(dataframes)
renewables = renewables.reset_index()

NameError: name 'transnetbw_df' is not defined

In [ ]:
# Only these columns will be kept for the renewable power plant list output
column_interest = ['commissioning_date','decommission_date','energy_source',
                   'electrical_capacity','thermal_capacity','voltage_level','tso',
                   'dso','dso_id','eeg_id','bnetza_id','federal_state',
                   'postcode','municipality_code','municipality','address',
                   'utm_zone','utm_east','utm_north','data_source']

In [ ]:
# Clean dataframe from columns other than specified above
renewables = renewables.loc[:, column_interest]
renewables.reset_index(drop=True, inplace=True)

**First look at DataFrame structure and format**

In [ ]:
renewables.info()

### 3.1.5 Translate values and harmonize energy source
Different German terms for energy source, energy source subtypes and voltage levels are translated and harmonized across the individual data sources. The value_translation_list is provided here as csv in the input folder. It is loaded in _2.3 Setup of translation dictionaries_.

In [ ]:
print('replacing..')
# Running time: some minutes. %time prints the time your computer required for this step
%time renewables.replace(value_dict, inplace=True)

**Separate and assign energy source and subtypes**

In [28]:
# Zusammenhang energy_source, energy_source_subtype und die Übersetzung darstellen
energy_source_dict

{'01 (HöS)': nan,
 '02 (HöS/HS)': nan,
 '03 (HS)': nan,
 '04 (HS/MS)': nan,
 '05 (MS)': nan,
 '06 (MS/NS)': nan,
 '07 (NS)': nan,
 'biomass': 'biomass',
 'gas_landfill': 'gas',
 'gas_mine': 'gas',
 'gas_sewage': 'gas',
 'geothermal': 'geothermal',
 'hydro': 'hydro',
 'solar': 'solar',
 'solar_ground': 'solar',
 'wind_offshore': 'wind',
 'wind_onshore': 'wind'}

In [ ]:
# Column energy_source first partly contains subtype information, thus subtype information
# are copied to new column for energy_source_subtype
renewables['energy_source_subtype'] = renewables['energy_source']

# and energy source subtype values in the energy_source column are replaced
renewables['energy_source'].replace(energy_source_dict, inplace=True)

**Summary of DataFrame**

In [ ]:
# Electrical capacity per energy_source (in MW)
renewables.groupby(['energy_source'])['electrical_capacity'].sum() / 1000

In [ ]:
# Electrical capacity per energy_source_subtype (in MW)
renewables.groupby(['energy_source_subtype'])['electrical_capacity'].sum() / 1000

### 3.1.6 Georeferencing

#### Get coordinates by postcode
*(for data with no existing geocoordinates)*

The available post code in the original data provides a first approximation for the geocoordinates of the RE power plants.<br>
The BNetzA data provides the full zip code whereas due to data privacy the TSOs only report the first three digits of the power plant's post code (e.g. 024xx) and no address. Subsequently a centroid of the post code region polygon is used to find the coordinates.

With data from
*  http://www.suche-postleitzahl.org/downloads?download=plz-gebiete.shp.zip
*  http://www.suche-postleitzahl.org/downloads?download_file=plz-3stellig.shp.zip
*  http://www.suche-postleitzahl.org/downloads

a CSV-file for all existing German post codes with matching geocoordinates has been compiled. The latitude and longitude coordinates were generated by running a PostgreSQL + PostGIS database. Additionally the respective TSO has been added to each post code. *(A Link to the SQL script will follow here later)*

*(License: http://www.suche-postleitzahl.org/downloads, Open Database Licence for free use. Source of data: © OpenStreetMap contributors)*

In [ ]:
# Read generated postcode/location file
postcode = pd.read_csv('input/de_tso_postcode_gps.csv',
                       sep=';',
                       header=0)

# Drop possible duplicates in postcodes
postcode.drop_duplicates('postcode', keep='last',inplace=True)

# Show first entries
postcode.head()

** Merge geometry information by using the postcode**

In [ ]:
# Take postcode and longitude/latitude informations
postcode = postcode[[0,3,4]]

renewables = renewables.merge(postcode, on=['postcode'],  how='left')

#### Transform geoinformation
*(for data with already existing geoinformation)*

In this section the existing geoinformation (in UTM-format) will be transformed into latidude and longitude coordiates as a uniform standard for geoinformation. 

The BNetzA data set offers UTM Geoinformation with the columns *utm_zone (UTM-Zonenwert)*, *utm_east* and *utm_north*. Most of utm_east-values include the utm_zone-value **32** at the beginning of the number. In order to properly standardize and transform this geoinformation into latitude and longitude it is necessary to remove this utm_zone value. For all UTM entries the utm_zone 32 is used by the BNetzA.


|utm_zone|	 utm_east|	 utm_north| comment|
|---|---|---| ----|
|32|	413151.72|	6027467.73| proper coordinates|
|32|	**32**912159.6008|	5692423.9664| caused error by 32|


**How many different utm_zone values are in the data set?**

In [ ]:
renewables.groupby(['utm_zone'])['utm_zone'].count()

**Remove the utm_zone "32" from the utm_east value**

In [ ]:
# Find entries with 32 value at the beginning
ix_32 = (renewables['utm_east'].astype(str).str[:2] == '32')
ix_notnull = renewables['utm_east'].notnull()

# Remove 32 from utm_east entries
renewables.loc[ix_32,'utm_east'] = renewables.loc[ix_32,'utm_east'].astype(str).str[2:].astype(float)

**Conversion UTM to lat/lon**

In [ ]:
# Convert from UTM values to latitude and longitude coordinates
try:
    renewables['lonlat'] = renewables.loc[ix_notnull, ['utm_east', 'utm_north', 'utm_zone']].apply(
        lambda x: utm.to_latlon(x[0], x[1], x[2], 'U'),
        axis=1) \
        .astype(str)
    
except:
    renewables['lonlat'] = np.NaN
    
lat = []
lon = []

for row in renewables['lonlat']:
    try:
        # Split tuple format into the column lat and lon  
        row = row.lstrip('(').rstrip(')')
        lat.append(row.split(',')[0])
        lon.append(row.split(',')[1])
    except:
        # set NaN 
        lat.append(np.NaN)
        lon.append(np.NaN)
          

renewables['latitude'] = lat
renewables['longitude'] = lon    

# Add new values to data frame lon and lat
renewables['lon'] = renewables[['longitude', 'lon']].apply(
    lambda x: x[1] if pd.isnull(x[0]) else x[0],
    axis=1)

renewables['lat'] = renewables[['latitude', 'lat']].apply(
    lambda x: x[1] if pd.isnull(x[0]) else x[0],
    axis=1)


**Check: missing coordinates by data source and type**

In [ ]:
print('Missing Coordinates ', renewables.lat.isnull().sum())

renewables[renewables.lat.isnull()].groupby(['energy_source',
                                             'data_source']
                                            )['data_source'].count()

### 3.1.7 Save
 
The merged, translated, cleaned, data frame will be saved temporily as a pickle file, which stores a python object fast.

In [ ]:
renewables.to_pickle('renewables.pickle')

## 3.2 Denmark DK

### 3.2.1 Download and read
The data which will be processed below is provided by the following data sources:

** [Energistyrelsen (ens) / Danish Energy Agency](http://www.ens.dk/info/tal-kort/statistik-noegletal/oversigt-energisektoren/stamdataregister-vindmoller)** - The wind turbines register is released by the Danish Energy Agency. 

** [Energinet.dk](http://www.energinet.dk/DA/El/Engrosmarked/Udtraek-af-markedsdata/Sider/Statistik.aspx)** - The data of solar power plants are released by the leading transmission network operator Denmark.

** [geonames.org](http://download.geonames.org/export/zip/?C=N;O=D)** The postcode  data from Denmark is provided by Geonames and licensed under a [Creative Commons Attribution 3.0 license](http://creativecommons.org/licenses/by/3.0/).

In [ ]:
# point URLs to original data depending on the chosen download option
if download_from == 'original_sources':
    
    url_DK_ens = 'http://www.ens.dk/sites/ens.dk/files/info/tal-kort/statistik-noegletal/oversigt-energisektoren/stamdataregister-vindmoeller/anlaegprodtilnettet.xls'
    url_DK_energinet = 'http://www.energinet.dk/SiteCollectionDocuments/Danske%20dokumenter/El/SolcelleGraf.xlsx'
    url_DK_geo = 'http://download.geonames.org/export/zip/DK.zip'

else:
    url_DK_ens = (url_opsd + version + folder + '/DK/anlaegprodtilnettet.xls')
    url_DK_energinet = (url_opsd + version + folder + '/DK/SolcelleGraf.xlsx')
    url_DK_geo = (url_opsd + version + folder + 'DK/DK.zip')

## 3.3 France FR

### 3.3.1 Download and read
The data which will be processed below is provided by the following data sources:

** [Ministery of the Environment, Energy and the Sea](http://www.statistiques.developpement-durable.gouv.fr/energie-climat/r/energies-renouvelables.html?tx_ttnews%5Btt_news%5D=24638&cHash=d237bf9985fdca39d7d8c5dc84fb95f9)** - Number of installations and installed capacity of the different renewable source for every municipality in France. Service of observation and statistics, survey, date of last update: 15/12/2015.

** [OpenDataSoft](https://public.opendatasoft.com/explore/?sort=modified)** A list of French INSEE codes and corresponding coordinates is published under the [Licence Ouverte (Etalab)](https://www.etalab.gouv.fr/licence-ouverte-open-licence).

In [ ]:
# point URLs to original data depending on the chosen download option
if download_from == 'original_sources':
    
    url_FR_gouv = "http://www.statistiques.developpement-durable.gouv.fr/fileadmin/documents/Themes/Energies_et_climat/Les_differentes_energies/Energies_renouvelables/donnees_locales/2014/electricite-renouvelable-par-commune-2014.xls"
    url_FR_geo = 'http://public.opendatasoft.com/explore/dataset/code-postal-code-insee-2015/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true'

else:
    url_FR_gouv = (url_opsd + version + folder + '/FR/electricite-renouvelable-par-commune-2014.xls')
    url_FR_geo = (url_opsd + version + folder + 'FR/code-postal-code-insee-2015.csv')

## 3.4 Poland PL

### 3.4.1 Download and read
The data which will be processed below is provided by the following data source:

** [Urzad Regulacji Energetyki (URE) / Energy Regulatory Office](http://www.ure.gov.pl/uremapoze/mapa.html)** - Number of installations and installed capacity per energy source of renewable energy. Summed per powiat (districts) .

In [ ]:
    url_PL_ure = (url_opsd + version + folder + '/PL/simple.rtf')

Check and validation of the renewable power plants list as well as the creation of CSV/XLSX/SQLite files can be found in Part 2 of this script. It also generates a daily timeseries of cumulated installed capacities by energy source.